In [1]:
import torch
from modulus.models.mlp.fully_connected import FullyConnected
model = FullyConnected(in_features=32, out_features=64)
input = torch.randn(128, 32)
output = model(input)
output.shape


torch.Size([128, 64])

In [3]:
import torch

import modulus
from modulus.datapipes.benchmarks.darcy import Darcy2D
from modulus.metrics.general.mse import mse
from modulus.models.fno.fno import FNO

Warp 1.1.0 initialized:
   CUDA Toolkit 11.5, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:1"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:2"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:3"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:4"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:5"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:6"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:7"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /root/.cache/warp/1.1.0


In [4]:
normaliser = {
    "permeability": (1.25, 0.75),
    "darcy": (4.52e-2, 2.79e-2),
}
dataloader = Darcy2D(
    resolution=256, batch_size=64, nr_permeability_freq=5, normaliser=normaliser
)
model = FNO(
    in_channels=1,
    out_channels=1,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=4,
    num_fno_modes=12,
    padding=5,
).to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda step: 0.85**step
)

# run for 20 iterations
for i in range(20):
    batch = next(iter(dataloader))
    true = batch["darcy"]
    pred = model(batch["permeability"])
    loss = mse(pred, true)
    loss.backward()
    optimizer.step()
    scheduler.step()

    print(f"Iteration:{i}. Loss:{loss.detach().cpu().numpy()}")

Module modulus.datapipes.benchmarks.kernels.initialization load on device 'cuda:0' took 512.85 ms
Module modulus.datapipes.benchmarks.kernels.utils load on device 'cuda:0' took 894.81 ms
Module modulus.datapipes.benchmarks.kernels.finite_difference load on device 'cuda:0' took 551.96 ms


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacity of 39.38 GiB of which 207.81 MiB is free. Including non-PyTorch memory, this process has 0 bytes memory in use. Of the allocated memory 377.03 MiB is allocated by PyTorch, and 12.97 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)